In [12]:
import argparse
import os
import shutil
from src.analysis.project_parser import parse_project
from src.analysis.dependency_resolver import resolve_dependencies
from src.analysis.feature_mapper import map_features_to_components
from src.generation.project_generator import generate_project
from src.generation.feature_integration import integrate_features

from src.models.prompt_templates import get_prompt_template
from src.utils.logger import logger
from src.utils.file_operations import read_file, write_file
from src.vector_database.db_builder import build_vector_database
from src.vector_database.db_query import query_vector_database

In [14]:
from src.models.llm_inference import answer_questions_from_dataframe
question = "How to integrate the specified features into the project components?"
persist_directory = "./notebooks/chroma_db"
collection_name = "my_vector_collection"
response = answer_questions_from_dataframe(question, collection_name, persist_directory)
print(response)


--------------------------------- Generated response -----------------------------------
{'query': 'How to integrate the specified features into the project components?', 'result': ' You can use the `integrate_features` function from the `feature_integration.py` module, which takes a list of project components and feature instructions as input and modifies the components accordingly. The function writes the feature instructions to the end of each component file. For example: `integrate_features(["component1.py", "component2.py"], "feature_instructions")`. \n\nNote: The `integrate_features` function does not return any value, it modifies the project components in-place. \n\nAlso, you can use the `map_features_to_components` function from the `feature_mapper.py` module to map feature requests to specific project components. This function takes parsed project data and a user-provided feature request as input and returns a mapping of features to components. \n\nFor example: `map_features_t

In [1]:
from src.models.llm_inference import query_llm
prompt = "Who is man's best friend?"
response = query_llm(prompt)
print(response)


Debug: Received prompt: Who is man's best friend?
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Who is man's best friend?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.
Debug: Response as string: That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.
That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with 

In [3]:
from src.vector_database.db_load import load_vector_db
from src.models.llm_inference import query_llm

prompt = "How to integrate the specified features into the project components?"
# Define the persistent directory and collection name
persist_directory = "./notebooks/chroma_db"
collection_name = "my_vector_collection"

# Load the vector database
vector_db = load_vector_db(collection_name, persist_directory)

# Query the vector database using the `query_llm` function
response = query_llm(user_input=prompt, vector_db=vector_db)

# Print the response
print("Generated Response:")
print(response)

Debug: Received prompt: How to integrate the specified features into the project components?
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
How to integrate the specified features into the project components?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nHow to integrate the specified features into the project components?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n', 'result': " \n\nTo integrate the specified features into the project components, you can use the `integrate_features` function from the `feature_integration.py` file. This function takes two arguments: `project_components` and `feature_instructions`. \n\nHere's how you can use it:\n\n1. First, you need to get the list of project components that you want to modify. You can do this by using the `map_features_to_components` function from the `feature_mapper.py` file, which maps feature requests to specific project components.\n\n2. Next, you need to get the feature instructions for the specified features. The format of these instructions is not specified in the 